In [1]:
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime
import json
import queue
import threading

import fundamentalanalysis as fa
import fa_mods as famods
import fa_utils

with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [2]:
all_companies = fa.available_companies(api_key)
tradeable_companies = famods.tradable_companies(api_key)

In [3]:
exchanges = all_companies.groupby('exchangeShortName')['exchange'].agg('unique').reset_index().rename(columns={'exchangeShortName': 'short_name', 'unique': 'exchange'})

In [4]:
datestamp = datetime.now().strftime('%y%m%d')

with open(f'../data/all_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(all_companies, file)

with open(f'../data/all_tradable_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(tradeable_companies, file)

with open(f'../data/exchanges_{datestamp}.pkl', 'wb') as file:
    pickle.dump(exchanges, file)

In [5]:
lse_companies = all_companies[(all_companies.exchangeShortName == 'LSE')]
nyse_companies = all_companies[(all_companies.exchangeShortName == 'NYSE')]
nasdaq_companies = all_companies[(all_companies.exchangeShortName == 'NASDAQ')]

lse_stocks = lse_companies[(lse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nyse_stocks = nyse_companies[(nyse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nasdaq_stocks = nasdaq_companies[(nasdaq_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

print(f"num LSE companies: {len(lse_companies)}")
print(f"num LSE stocks: {len(lse_stocks)}")
print()
print(f"num NYSE companies: {len(nyse_companies)}")
print(f"num NYSE stocks: {len(nyse_stocks)}")
print()
print(f"num NASDAQ companies: {len(nasdaq_companies)}")
print(f"num NASDAQ stocks: {len(nasdaq_stocks)}")

num LSE companies: 6335
num LSE stocks: 3877

num NYSE companies: 4070
num NYSE stocks: 3463

num NASDAQ companies: 10249
num NASDAQ stocks: 5916


In [6]:
thread_lock = threading.Lock()

def get_exchange_data(exchange: str):
    exchange_companies = all_companies[(all_companies.exchangeShortName == exchange)]
    exchange_stocks = exchange_companies[(exchange_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

    symbols = exchange_stocks.symbol.tolist()
    stock_data = {}
    failed = []
    num_stocks = len(symbols)
    cnt = 0

    symbol_queue = queue.Queue()
    [symbol_queue.put(s) for s in symbols]
    symbol_queue.qsize()

    def worker():
        nonlocal cnt
        nonlocal stock_data
        nonlocal failed
        while True:
            with thread_lock:
                cnt += 1
                symbol = symbol_queue.get()        
                print(f'{exchange} [{cnt}/{num_stocks}]: getting {symbol} - f[{len(failed)}], s[{len(stock_data)}]')

            if symbol in stock_data:
                print(f"Already have {symbol}, skipping")
                continue

            try:
                data = fa_utils.get_ticker_data(symbol, api_key)
                stock_data[symbol] = data
                print(f"{symbol} SUCCESS")
            except:
                print(f"{symbol} FAILED")
                failed.append(symbol)

            symbol_queue.task_done()

    for x in range(4):
        name = "Thread_" + str(x)
        t = threading.Thread(name=name,target=worker, daemon=True).start()

    symbol_queue.join()

    print(len(failed))
    print(len(stock_data))

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'wb') as file:
        pickle.dump(stock_data, file)

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}_failed.pkl', 'wb') as file:
        pickle.dump(failed, file)

    return stock_data, failed

In [7]:
stock_data, failed = get_exchange_data('LSE')

1 Working on HON.L - f[0], s[0]
2 Working on AIR3.L - f[0], s[0]
3 Working on FB1.L - f[0], s[0]
4 Working on 2UKS.L - f[0], s[0]
AIR3.L FAILED
5 Working on MRNS.L - f[1], s[0]
FB1.L FAILED
6 Working on EMUM.L - f[2], s[0]
2UKS.L FAILED
7 Working on 72HN.L - f[3], s[0]
EMUM.L FAILED
8 Working on PMGZ.L - f[4], s[0]
MRNS.L FAILED
9 Working on MNDI.L - f[5], s[0]
72HN.L FAILED
10 Working on 0VOU.L - f[6], s[0]
PMGZ.L FAILED0VOU.L FAILED
11 Working on PRES.L - f[7], s[0]

12 Working on IUG.L - f[8], s[0]
IUG.L FAILED
13 Working on ITH.L - f[9], s[0]
ITH.L FAILED
14 Working on SENS.L - f[10], s[0]
HON.L FAILED
15 Working on IQG.L - f[11], s[0]
SENS.L FAILED
16 Working on FEML.L - f[12], s[0]
FEML.L FAILED
17 Working on 5QQE.L - f[13], s[0]
5QQE.L FAILED
18 Working on 0RHR.L - f[14], s[0]
0RHR.L FAILED
19 Working on SFBE.L - f[15], s[0]
SFBE.L FAILED
20 Working on 3LBA.L - f[16], s[0]
3LBA.L FAILED
21 Working on CCPE.L - f[17], s[0]
CCPE.L FAILED
22 Working on 0QL7.L - f[18], s[0]
0QL7.L FA

KeyboardInterrupt: 

BIFF.L SUCCESS
48 Working on MCRO.L - f[38], s[6]
NUM.L SUCCESS
49 Working on RBD.L - f[38], s[7]


In [ ]:
exchanges = ["NYSE", "LSE"]

for exchange in exchanges:

    exchange_companies = all_companies[(all_companies.exchangeShortName == exchange)]
    exchange_stocks = exchange_companies[(exchange_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

    stock_data = {}
    failed = []

    symbol_queue = queue.Queue()
    [symbol_queue.put(s) for s in exchange_stocks.symbol.tolist()]
    symbol_queue.qsize()

    def worker():
        while True:
            symbol = symbol_queue.get()
            print(f'Working on {symbol}')

            if symbol in stock_data:
                print(f"Already have {symbol}, skipping")
                continue

            try:
                data = fa_utils.get_ticker_data(symbol, api_key)
                stock_data[symbol] = data
                print(f"{symbol} SUCCESS")
            except:
                print(f"{symbol} FAILED")
                failed.append(symbol)

            symbol_queue.task_done()

    for x in range(4):
        name = "Thread_" + str(x)
        t = threading.Thread(name=name,target=worker, daemon=True).start()

    symbol_queue.join()

    print(len(failed))
    print(len(stock_data))

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'wb') as file:
        pickle.dump(stock_data, file)

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}_failed.pkl', 'wb') as file:
        pickle.dump(failed, file)

In [ ]:
print(len(failed))
print(len(stock_data))

In [ ]:
with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'wb') as file:
    pickle.dump(stock_data, file)

with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}_failed.pkl', 'wb') as file:
    pickle.dump(failed, file)

In [ ]:
datestamp

In [ ]:
with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'rb') as file:
    stock_data_2 = pickle.load(file)

print(len(stock_data_2))
display(stock_data_2)